# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [1]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [2]:
def VLSR(solar_radius, mu=6.379, v_sun=12.24*u.km/u.s):
    """
    This function will compute the velocity at the local standard of rest
        VLSR = 4.74*mu*Ro - vsun
    Inputs: Ro (astropy units kpc) Distance from the sun to the galactic center,
            mu is the proper motion of Sag A* (mas/yr)
                Default is from Reid & Brunthaler 2004
            vsun (astropy units km/s) the peculiar motion of the sun
                in the v direction (Schonrich+2010)

    Outputs: VLSR (astropy units km/s) the local standard of rest
    """
    VLSR = 4.74*mu*(solar_radius/u.kpc)*u.km/u.s - v_sun
    return VLSR

In [3]:
# Different values of the distance to the Galactic Center
RoReid = 8.34*u.kpc # Reid + 2014
RoAbuter = 8.178*u.kpc # GRAVITY Abuter + 2019
RoSparke = 7.9*u.kpc # Sparke & Gallagher Text

In [10]:
# Computer VLSR using Reid 2014
VLSR_Reid = VLSR(RoReid)
VLSR_Reid

<Quantity 239.9320764 km / s>

In [11]:
# Compute VLSR using GRAVITY Collab
VLSR_Abuter = VLSR(RoAbuter)
VLSR_Abuter

<Quantity 235.03376988 km / s>

In [13]:
# Copmute VLSR using Sparke & Gallagher
VLSR_Sparke = VLSR(RoSparke)
VLSR_Sparke

<Quantity 226.628034 km / s>

### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [14]:
# Orbital Period = 2piR/Vel

def TorbSun(Ro, Vc):
    """
    This function calculates the orbital period of the sun.
        T = 2 pi R / V

    Inputs: R_sun (astropy quantity) distance to galactic center from the sun (kpc)
            Vc (astropy quantity) velocity of the sun in the "v" direction (km/s)

    Outputs: T (astropy quanity) Orbital period (Gyr)
    """
    
    VkpcGyr = Vc.to(u.kpc/u.Gyr) # Converting V to kpc/Gyr
    T = 2 * np.pi * Ro/VkpcGyr   # Orbital period
    
    return T

In [18]:
VsunPec = 12.24*u.km/u.s # Peculiar Motion

In [19]:
Vsun = VLSR_Abuter + VsunPec # Total motion of the sun in the "v" direction

In [21]:
# Orbital Period of the Sun
T_Abuter = TorbSun(RoAbuter, Vsun)
np.around(T_Abuter, 3)

<Quantity 0.203 Gyr>

### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [24]:
AgeUniverse = 13.8 * u.Gyr
AgeUniverse/T_Abuter

<Quantity 67.91779593>

## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [29]:
print(const.G)

  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018


In [31]:
Grav = const.G.to(u.kpc**3/u.Gyr**2/u.Msun)
Grav

<Quantity 4.49850215e-06 kpc3 / (solMass Gyr2)>

In [32]:
# Denisty profile rho = VLSR^2 / (4 * pi * G * r^2)
# Mass (r) = Integrate rho dV
#            Integrate rho 4*pi*r^2*dr
#            Integrate VLSR^2 / (4*pi*G*r^2) * 4 * pi * r^2 dr
#            VLSR^2/G * r

def massIso(r, VLSR):
    """
    This function will compute the dark matter mass enclosed in a sphere of radius r.
    Assuming an Isothermal Sphere Model.
        M(r) = VLSR^2/G * r

    Inputs: r (astropy quantity), distance from the Galactic Center (kpc)
            VLSR (astropy quantity), velocity at local standard of rest (km/s)

    Outputs: M (astropy quantity), mass enclosed withinn r (Msun)
    """
    
    VLSRkpcGyr = VLSR.to(u.kpc/u.Gyr) # Converting to kpc/Gyr

    M = (VLSRkpcGyr**2 / Grav) * r   # Isothermal Mass Profile
    
    return M

In [36]:
# Compute the mass enclosed within Ro (GRAVITY Collab)
mIsoSolar = massIso(RoAbuter, VLSR_Abuter) # Mass of sphere of radius solar distance using Isothermal Mass Profile
mIsoSolar

<Quantity 1.05038026e+11 solMass>

In [37]:
# Compute the mass enclosed within 260 kpc
mIso260 = massIso(260*u.kpc, VLSR_Abuter)
mIso260

<Quantity 3.33943345e+12 solMass>

## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [42]:
# Potential for a Hernquist Sphere
# Phi = -G*m/(r+a)

# Escape Speed becomes:
# vesc^2 = 2*G*M/(r+a)

# Rearrange for M
# M = vesc^2/2/G*(r+a)

def massHernVesc(vesc, r, a=30*u.kpc):
    """
    This function computes the total mass needed given an escape speed, assuming a Hernquist profile
        M = vesc^2/2/G*(r+a)

    Inputs: vesc (astropy quantity), escape speed (or speed of satellite) (km/s)
            r (astropy quantity), distance from the Galactic Center (kpc)
            a (astropy quantity), the Hernquist Scale length (kpc)
                                  default value of 30kpc

    Outpus: M (astropy quantity), mass within r (Msun)
    """

    vescKpcGyr = vesc.to(u.kpc/u.Gyr) # Convert to kpc/Gyr

    M = vescKpcGyr**2 / 2 / Grav * (r + a)

    return M

In [43]:
Vleo = 196*u.km/u.s # Speed of Leo I Sohn et al.
r = 260*u.kpc

In [44]:
MLeoI = massHernVesc(Vleo, r)
MLeoI

<Quantity 1.29514698e+12 solMass>

In [46]:
mIso260/MLeoI

<Quantity 2.57842045>